## Deep Learning Project: Build a Traffic Sign Recognition Classifier
This project explores the deep learning process, making use of TensorFlow to build a model (similar to the LeNet model) that is able to classify traffic signs with up to ~94% accuracy.

In five main steps:
* Step 0: Load the Data
* Step 1: Data Summary & Exploration
* Step 2: Design and Test a Model Architecture
* Step 3: Test the Model on New images
* Step 4: Visualize the Neural Network's State with Test Images

## Step 0: Load the Data

In [ ]:
# Load pickled data
import pickle

# Load, open, and assign training, validation, and testing data for German traffic signs

# File address
training_file = 'traffic_signs_data/train.p'
validation_file = 'traffic_signs_data/valid.p'
testing_file = 'traffic_signs_data/test.p'

# Opening files
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

# Assigning features and label data (input and output)
X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---
## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary, but the two that will be used are the features (inputs) and labels (outputs)
- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.

### Dataset Summary

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results
import numpy as np

# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_validation)

# Number of testing examples.
n_test = len(X_test)

# Shape of traffic sign images
image_shape = X_train[0].shape

# Number of unique classes/labels
n_classes = np.amax(y_train) - np.amin(y_train) + 1

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Exploratory Visualization

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Choose a random image from the data set
index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

# Plot the random image and display the label as well
plt.figure(figsize=(1,1))
plt.imshow(image)
plt.show()
print(y_train[index])

# Create histograms of labels for each data set
train_hist, train_edges = np.histogram(y_train, bins = range(43))
valid_hist, valid_edges = np.histogram(y_validation, bins = range(43))
test_hist, test_edges = np.histogram(y_test, bins = range(43))

# Plot histograms
plt.bar(train_edges[:-1], train_hist, width = 1)
plt.title('Training Data')
plt.xlim(min(train_edges), max(train_edges))
plt.show()
plt.bar(valid_edges[:-1], valid_hist, width = 1)
plt.title('Validation Data')
plt.xlim(min(valid_edges), max(valid_edges))
plt.show()
plt.bar(test_edges[:-1], test_hist, width = 1)
plt.title('Testing Data')
plt.xlim(min(valid_edges), max(valid_edges))
plt.show()

----
## Step 2: Design and Test the Model Architecture

### Pre-process the Data Set (normalization, grayscale, etc.)

In [ ]:
# Obtain teh shuffle to randomize the batches for each run
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

# Normalize the data of 8 bits to be between [-1,1) instead of [0,255]
X_train = X_train/128 - 1
X_validation = X_validation/128 - 1
X_test = X_test/128 - 1

### Model Architecture

In [ ]:
import tensorflow as tf

# Set some of the Hyperparameters
EPOCHS = 15
BATCH_SIZE = 32
keep_prob = tf.placeholder(tf.float32)

from tensorflow.contrib.layers import flatten

# Create LeNet-adjusted architecture
def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Weights creates as a dictionary using the truncated_normal using below input shapes
    weights = {
        'wc1': tf.Variable(tf.truncated_normal(shape = (5, 5, 3, 6), mean = mu, stddev = sigma)),
        'wc2': tf.Variable(tf.truncated_normal(shape = (5, 5, 6, 16), mean = mu, stddev = sigma)),
        'wd1': tf.Variable(tf.truncated_normal(shape = (400, 120), mean = mu, stddev = sigma)),
        'wd2': tf.Variable(tf.truncated_normal(shape = (120, 84), mean = mu, stddev = sigma)),
        'out': tf.Variable(tf.truncated_normal(shape = (84, 43), mean = mu, stddev = sigma))
    }

    biases = {
        'bc1': tf.Variable(tf.zeros(6)),
        'bc2': tf.Variable(tf.zeros(16)),
        'bd1': tf.Variable(tf.zeros(120)),
        'bd2': tf.Variable(tf.zeros(84)),
        'out': tf.Variable(tf.zeros(43))
    }
    
    # Set strides to be 1
    strides_set = [1, 1, 1, 1]

    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    layer1 = tf.nn.conv2d(x,weights['wc1'], strides=strides_set, padding='VALID')
    layer1 = tf.add(layer1,biases['bc1'])
    # ReLU Activation
    layer1 = tf.nn.relu(layer1)
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    k_size = [1, 2, 2, 1]
    pool_stride = [1, 2, 2, 1]
    pool1 = tf.nn.max_pool(layer1, ksize=k_size, strides=pool_stride, padding='VALID')
    
    # Layer 2: Convolutional. Output = 10x10x16.
    layer2 = tf.nn.conv2d(pool1,weights['wc2'], strides=strides_set, padding='VALID')
    layer2 = tf.add(layer2,biases['bc2'])    
    # ReLU Activation
    layer2 = tf.nn.relu(layer2)
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    pool2 = tf.nn.max_pool(layer2, ksize=k_size, strides=pool_stride, padding='VALID')
                           
    # Flatten. Input = 5x5x16. Output = 400.
    flatten = tf.reshape(pool2, [-1, weights['wd1'].get_shape().as_list()[0]])
                           
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = tf.add(tf.matmul(flatten,weights['wd1']),biases['bd1'])
    # ReLU Activation and dropout
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = tf.add(tf.matmul(fc1,weights['wd2']),biases['bd2'])
    # ReLU Activation and Dropout
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, keep_prob)

    # Layer 5: Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(fc2,weights['out']),biases['out'])
    
    return logits, layer1, pool1, layer2, pool2, fc1, fc2

### Features and Labels
`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.
One-hot encoding 

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

### Train, Validate and Test the Model

#### Training Pipeline

In [ ]:
# Set Hyperparameter learning rate
rate = 0.0005

# Output all layers of CNN, then set cross entropy using softmax, loss calculation, AdamOptimizer, and training operation
logits, layer1, pool1, layer2, pool2, fc1, fc2 = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

#### Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

In [ ]:
# Compared the maximum softmax prediction to the label, check if the label is correct
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
# Calculate the accuracy of the prediction
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the model saving function
saver = tf.train.Saver()

# Create evaluate function for testing purposes
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob:1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

#### Train the Model
Run the training data through the training pipeline to train the model.
* Before each epoch, shuffle the training set.
* After each epoch, measure the loss and accuracy of the validation set.
* Save the model after training.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.6})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        training_accuracy = evaluate(X_train, y_train)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

## Evaluate the Model
The model is run one time using the testing batch of german traffic signs, providing a more accurate final accuracy % of classifying traffic signs.

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---
## Step 3: Test the Model on New Images

Five pictures of German traffic signs were obtained from a Google image search, and the model is run to test them.

### Load and Output the Images

In [ ]:
import os
import matplotlib.image as mpimg
# Load the images from the local folder
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = mpimg.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
filepath = 'new_images/'
images = load_images(filepath)

# Show the five images
fig = plt.figure(figsize=(5, 5))  # width, height in inches

fig,ax = plt.subplots(1,5)
for i in range(5):
    ax[i].imshow(images[i])

### Predict the Sign Type for Each Image
Labels should be (in order)

* 13: Yield
* 34: Turn left ahead
* 40: Roundabout mandatory
* 36: Go straight or right
* 12: Priority road

In [ ]:
for i in range(5):
    images[i] = images[i]/128 - 1
images = np.asarray(images)
images_labels = np.transpose([13, 34, 40, 36, 12])

In [ ]:
# The session is re-initialize and the model is loaded. 
# Then both the predictions and comparisons to the correct label is given
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    prediction = sess.run(tf.argmax(logits, 1), feed_dict={x: images, y: images_labels, keep_prob:1})
    print(prediction)
    compare = sess.run(correct_prediction, feed_dict={x: images, y: images_labels, keep_prob:1})
    print(compare)

### Analyze Performance

In [ ]:
# The accuracy is calculated for the five new images (out of 1)
overall_accuracy = sum(compare)/5
print(overall_accuracy)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For the five images below, `the tf.nn.top_k` function finds the top 5 softmax confidence calculation for each sign.
The assoiated indices (labels) are provided in the exact same order (most likle to least likely)

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    top_5 = sess.run(tf.nn.softmax(logits, 1), feed_dict={x: images, y: images_labels, keep_prob:1})
    top_softmax = sess.run(tf.nn.top_k(tf.constant(top_5), k=5))
    print(top_softmax)

---
## Step 4: Visualize the Neural Network's State with Test Images

Visualizing the create CNN's state at different layers of the model below using feature maps.

In [ ]:
# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

# Specifically, only the second layer is observed as it contains more details
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    sess.run(outputFeatureMap(images, layer2))